In [10]:
import secrets

In [11]:
def inverseMod(a, mod):
    return pow(a, -1, mod)

In [29]:
class Curve:
    def __init__(self, a, b, p):
        # curve configuration
        # y^2 = x^3 + a*x + b
        self.a = a
        self.b = b
        self.p = p
        
defaultCurve = Curve(a=0, b=7, p=23)

In [30]:
class Point:
    def __init__(self, x, y, curve=defaultCurve):
        xp = x % curve.p
        yp = y % curve.p
        
        while(xp < 0):
            xp = xp + curve.p

        while(yp < 0):
            yp = yp + curve.p
        
        self.x = xp
        self.y = yp
        self.curve = curve
        
    def copy(self):
        return Point(self.x, self.y, self.curve)
    
    def __neg__(self):
        return Point(self.x, -self.y, self.curve)
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y and self.curve == other.curve
        
    def __add__(self, other):
        if other.curve != self.curve:
            raise Exception("Can not add with different curves")
        
        if self == other:
            beta = (3 * self.x * self.x + self.curve.a) * inverseMod(2 * self.y, self.curve.p)
        else:
            beta = (other.y - self.y) * inverseMod(other.x - self.x, self.curve.p)

        xnew = beta * beta - self.x - other.x
        ynew = beta * (self.x - xnew) - self.y

        return Point(xnew, ynew, self.curve)
    
    def __sub__(self, other):
        return self + (-other)
    
    def __mul__(self, k):
        temp = self.copy()
        kAsBinary = bin(k)
        kAsBinary = kAsBinary[2:len(kAsBinary)]

        for i in range(1, len(kAsBinary)):
            temp = temp + temp
            if kAsBinary[i: i+1] == '1':
                temp = temp + self

        return temp
    
    def __rmul__(self, k):
        return self * k

g = Point(x=2, y=24)

## ECDH

In [31]:
# Alice gera publica
alicePrivate = secrets.randbelow(g.curve.p)
alicePublic = alicePrivate * g
print(f'{ alicePublic.x = }')

 alicePublic.x = 16


In [32]:
# Bob gera publica
bobPrivate = secrets.randbelow(g.curve.p)
bobPublic = bobPrivate * g
print(f'{ bobPublic.x = }')

 bobPublic.x = 6


In [33]:
# Alice computa shared a partir de publica de Bob
aliceShared = alicePrivate * bobPublic
print(f'{ aliceShared.x = }')

 aliceShared.x = 20


In [34]:
# Bob computa shared a partir de publica de Alice
bobShared = bobPrivate * alicePublic
print(f'{ bobShared.x = }')

 bobShared.x = 20


## Proposed

### Registration (gerar chaves publicas)

In [35]:
# Alice gera publica
alicePrivate = secrets.randbelow(g.curve.p)
alicePublic = alicePrivate * g
print(f'{ alicePublic.x = }')

 alicePublic.x = 10


In [36]:
# Bob gera publica
bobPrivate = secrets.randbelow(g.curve.p)
bobPublic = bobPrivate * g
print(f'{ bobPublic.x = }')

 bobPublic.x = 9


### Computacao (segredo da sessao e envio)

In [37]:
aliceSessionSecret = secrets.randbelow(g.curve.p)
toBob = (alicePrivate + aliceSessionSecret) * bobPublic
print(f'{ toBob.x = }')

 toBob.x = 9


In [38]:
bobSessionSecret = secrets.randbelow(g.curve.p)
toAlice = (bobPrivate + bobSessionSecret) * alicePublic
print(f'{ toAlice.x = }')

 toAlice.x = 0


### Computacao chave sessao

In [39]:
aliceSharedPartial = inverseMod(alicePrivate, g.curve.p) * toAlice - alicePublic
print(f'{ aliceSharedPartial.x = }')

aliceShared = aliceSharedPartial + (aliceSessionSecret * g)
print(f'{ aliceShared.x = }')

 aliceSharedPartial.x = 2
 aliceShared.x = 10


In [40]:
bobSharedPartial = inverseMod(bobPrivate, g.curve.p) * toBob - bobPublic
print(f'{ bobSharedPartial.x = }')

bobShared = bobSharedPartial + (bobSessionSecret * g)
print(f'{ bobShared.x = }')

 bobSharedPartial.x = 9
 bobShared.x = 0
